In [1]:
!git clone https://github.com/kr-viku/GLAUCOMA-DETECTION.git

Cloning into 'GLAUCOMA-DETECTION'...
remote: Enumerating objects: 563, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 563 (delta 32), reused 97 (delta 31), pack-reused 465
Receiving objects: 100% (563/563), 61.73 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (567/567), done.


['test', 'train']


In [23]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Define the model
classifier = Sequential([
    Conv2D(32, (3, 3), input_shape=(256, 256, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define a function to load and preprocess image files
def preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    return image, label

# Define paths to the training and testing data directories
train_data_dir = '/content/GLAUCOMA-DETECTION/dataset/train'
test_data_dir = '/content/GLAUCOMA-DETECTION/dataset/test'

# Map class names to numerical labels
class_to_label = {'class0': 0, 'class1': 1}

# Get a list of all image filenames and their corresponding labels for training data
train_filenames = []
train_labels = []
for class_name in os.listdir(train_data_dir):
    class_dir = os.path.join(train_data_dir, class_name)
    class_label = class_to_label[class_name]
    for fname in os.listdir(class_dir):
        train_filenames.append(os.path.join(class_dir, fname))
        train_labels.append(class_label)

# Get a list of all image filenames and their corresponding labels for testing data
test_filenames = []
test_labels = []
for class_name in os.listdir(test_data_dir):
    class_dir = os.path.join(test_data_dir, class_name)
    class_label = class_to_label[class_name]
    for fname in os.listdir(class_dir):
        test_filenames.append(os.path.join(class_dir, fname))
        test_labels.append(class_label)

# Create TensorFlow datasets from the image filenames and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))

# Map the preprocess_image function to the datasets
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)

# Fit the model to the training data
model_info = classifier.fit(
    train_dataset,
    epochs=3,
    validation_data=test_dataset
)


Epoch 1/3
15/15 [==============================] - 45s 3s/step - loss: 1.2314 - accuracy: 0.5714 - val_loss: 0.6465 - val_accuracy: 0.6250
Epoch 2/3
15/15 [==============================] - 39s 3s/step - loss: 0.5982 - accuracy: 0.6593 - val_loss: 0.5942 - val_accuracy: 0.6719
Epoch 3/3
15/15 [==============================] - 38s 3s/step - loss: 0.5319 - accuracy: 0.7714 - val_loss: 0.5897 - val_accuracy: 0.6719
